In [2]:
import pandas as pd

## Biomass

In [23]:
biomass = pd.read_excel('Biomass.xls')

In [24]:
#no repeated FIPS
biomass = biomass[['FIPS','Total']]

In [68]:
biomass

,FIPS,Total
0,51063,22617.0
1,51141,104430.0
2,51121,90447.0
3,51045,8634.3
4,51005,195020.0
...,...,...
129,51550,73576.0
130,51115,6748.6
131,51650,24107.0
132,51740,16178.0


## Water Availability

In [25]:
water_avail = pd.read_excel('Water_Avail.xls')

In [26]:
water_avail = water_avail[['FIPS','frsh_chg_Mton']]

In [27]:
#rename column
water_avail = water_avail.rename(columns={'frsh_chg_Mton':'Water Availability'})

In [21]:
water_avail

,FIPS,Water Availability
0,51001,84.710667
1,51003,606.669853
2,51005,322.146458
3,51007,77.834088
4,51009,558.141032
...,...,...
128,51800,1115.128307
129,51810,122.122897
130,51820,6.652010
131,51830,17.100823


## Powerplant

In [3]:
power = pd.read_excel('Powerplant.xlsx')

In [4]:
#drop first row
power = power.drop(power.index[0])

In [5]:
#change fips to string
power['Plant FIPS state code'] = power['Plant FIPS state code'].astype("str")
power['Plant FIPS county code'] = power['Plant FIPS county code'].astype("str")

In [6]:
power[['Plant FIPS state code','Plant FIPS county code']].dtypes

Plant FIPS state code     object
Plant FIPS county code    object
dtype: object

In [7]:
#combine state and county code for full fips code
power['FIPS'] = power['Plant FIPS state code'] + power['Plant FIPS county code']

In [124]:
power1 = power[['FIPS','Plant primary fuel category']]

In [126]:
#0 if gas/oil/coal/nuclear (non-renewable), 1 else
power1['Renewable'] = power1['Plant primary fuel category'].apply(lambda x: 1 if x not in ['GAS', 'OIL', 'COAL', 'NUCLEAR'] else 0)

C:\Users\daven\AppData\Local\Temp\ipykernel_24144\2962015886.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  power1['Renewable'] = power1['Plant primary fuel category'].apply(lambda x: 1 if x not in ['GAS', 'OIL', 'COAL', 'NUCLEAR'] else 0)


In [99]:
power

,FIPS,Plant primary fuel category,Renewable
1,51031,BIOMASS,1
2,51031,SOLAR,1
3,51007,BIOMASS,1
4,51179,BIOMASS,1
5,51017,HYDRO,1
...,...,...,...
179,51143,SOLAR,1
180,51109,SOLAR,1
181,51191,GAS,0
182,51093,SOLAR,1


In [128]:
#groupby FIPS and sum over renewable column
power1 = power1.groupby('FIPS').agg({'Renewable':'sum'})

In [129]:
#anything > 0 is assigned 1, meaning it a renewable energy powerplant in county 
power1 = power1['Renewable'].apply(lambda x: 0 if x == 0 else 1)

In [130]:
power1 = power1.to_frame(name="Renewable")

In [131]:
power1 = power1.reset_index()

In [132]:
power1

,FIPS,Renewable
0,51001,1
1,51005,0
2,51007,1
3,51009,1
4,51011,1
...,...,...
78,51740,1
79,51760,0
80,51775,0
81,51800,1


In [55]:
#subset df
power2 = power[['FIPS','Plant annual total renewables net generation (MWh)']]

In [56]:
power2.sort_values(by='Plant annual total renewables net generation (MWh)')

,FIPS,Plant annual total renewables net generation (MWh)
5,51017,-750348
151,51143,-21344
91,51033,0
118,51019,0
117,51001,0
...,...,...
142,51101,310567.878
153,51175,317888.49
1,51031,321570.31
89,51117,523828


In [57]:
#modify negative values (they should be positive)
power2['Plant annual total renewables net generation (MWh)'].iloc[4] = -power2['Plant annual total renewables net generation (MWh)'].iloc[4]
power2['Plant annual total renewables net generation (MWh)'].iloc[150] = -power2['Plant annual total renewables net generation (MWh)'].iloc[150]

C:\Users\daven\AppData\Local\Temp\ipykernel_24832\1504794057.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  power2['Plant annual total renewables net generation (MWh)'].iloc[4] = -power2['Plant annual total renewables net generation (MWh)'].iloc[4]
C:\Users\daven\AppData\Local\Temp\ipykernel_24832\1504794057.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  power2['Plant annual total renewables net generation (MWh)'].iloc[150] = -power2['Plant annual total renewables net generation (MWh)'].iloc[150]


In [58]:
#drop 123 row (b/c na)
power2 = power2.drop(power2.index[122])

In [59]:
#sum all (renewable) energy per county
power2 = power2.groupby('FIPS').agg({'Plant annual total renewables net generation (MWh)':'sum'})

In [60]:
power2 = power2.reset_index()

In [61]:
power2['FIPS'] = power2['FIPS'].astype('str')

## Merge

In [62]:
new = pd.merge(biomass,water_avail,on='FIPS',how='left')

In [63]:
new['FIPS'] = new["FIPS"].astype('str')

In [64]:
new = pd.merge(new,power2,on="FIPS",how='left')

In [88]:
new

,FIPS,Total,Water Availability,Plant annual total renewables net generation (MWh)
0,51063,22617.0,405.545088,NaN
1,51141,104430.0,474.077209,NaN
2,51121,90447.0,336.533725,0
3,51045,8634.3,284.574781,NaN
4,51005,195020.0,322.146458,0
...,...,...,...,...
129,51550,73576.0,672.119754,74180
130,51115,6748.6,5.507963,NaN
131,51650,24107.0,84.906146,35869
132,51740,16178.0,178.002837,55723.183


## Old Data

In [66]:
#load data
scaled = pd.read_csv('data-scaled-agg.csv')

In [70]:
#rename county to FIPS
scaled = scaled.rename({"County":"FIPS"}, axis=1)

In [74]:
#make FIPS a string
scaled['FIPS'] = scaled['FIPS'].astype('str')

In [75]:
#merge and rename column
new_new = pd.merge(scaled,new,on="FIPS",how="left")
new_new = new_new.rename({'Plant annual total renewables net generation (MWh)':'Power',
                         'Total':'Biomass'},axis=1)

In [92]:
#check for nas. We have 26 nas for powerplant data
new_new['Power'].isnull().sum()

26